# Explore fixations and protocols together with location data
- synchronize fixations, protocols and GPS measurements
- plotting them on maps and explore them interactively


## Syncrhonize fixations and GPS
Using files:  
- data/grouped_screen_replaced_mapped_fixation.tsv
- gps.csv   

Syncrhonization logic: for every GPS point, the most frequent fixation target during that measurement period (1s) is associated with it. 

Result file: data/sync_fixation_gps.tsv

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
et = pd.read_csv('data/grouped_screen_replaced_mapped_fixation.tsv', sep='\t')
gps = pd.read_csv('data/gps.csv')

# sync it using your own marker. Here I know the ET segment starts at the fifth GPS measurement so I just dropped the first four.

# drop first four, start with five
gps = gps.iloc[4:].reset_index()

# also use your own marker
# GPS[0] -> video start time : 119694
gps['timestamp'] = (gps['index'] - 4)*1000 + 119694
gps['start'] = gps['timestamp'] - 500
gps['end'] = gps['timestamp'] + 500

# just map by the center of the fixation
for i in gps.index:
    try:
        # first et row within limits
        et_index_start = et[et['Recording timestamp'].ge(gps.at[i, 'start'])].index[0]
        # last et row within limits
        et_index_end = et[et['Recording timestamp'].le(gps.at[i, 'end'])].index[-1]
        target_list = []
        for j in range(et_index_start, et_index_end+1):
            target_list.append(et.at[j, 'new_target'])
        gps.at[i, 'new_target'] = target_list
    except:
        continue

# nan and empty lists
for row in gps.loc[gps.new_target.isnull(), 'new_target'].index:
    gps.at[row, 'new_target'] = []

# count and majority
gps['target_count'] = gps.new_target.str.len()
gps['target_maj'] = gps.new_target.apply(lambda x: stats.mode(x)[0][0] if len(x)!=0 else np.nan)

gps.to_csv('data/sync_fixation_gps.tsv', index=False, sep='\t')

## plot fixaton and gps together on a map
It would be easier explored in a GIS (e.g. QGIS). But anyway, here is an example of plotting it on a plot.ly interactive map.   

**For this, you need a mapbox token.**

In [3]:
import plotly.graph_objects as go 

In [9]:
df = pd.read_csv('data/sync_fixation_gps.tsv', sep='\t')
df.target_maj.fillna('No fixation', inplace=True)
df.target_maj.replace(['others', 'No fixation'], 'others / no fixation', inplace=True)
df.head()

,index,lat,lon,time,timestamp,start,end,new_target,target_count,target_maj
0,4,52.223773,6.886489,2020-04-27T11:20:58Z,119694,119194,120194,surroundings,12,surroundings
1,5,52.223773,6.886489,2020-04-27T11:20:59Z,120694,120194,121194,"['map_ar', 'surroundings']",2,map_ar
2,6,52.223773,6.886489,2020-04-27T11:21:00Z,121694,121194,122194,['map_ar'],1,map_ar
3,7,52.223773,6.886489,2020-04-27T11:21:01Z,122694,122194,123194,[],0,others / no fixation
4,8,52.223773,6.886489,2020-04-27T11:21:02Z,123694,123194,124194,['map_ar'],1,map_ar


In [10]:
color_dict = {'info': 'rgb(253, 180, 98)',
              'map_ar': 'rgb(144, 211, 199)',  # green-ish
              'old_map': "rgb(255, 237, 111)", # yellow-ish
              'take_note': 'rgb(252, 205, 229)',
              'view_note': 'rgb(190, 186, 218)',
              'building': 'rgb(128, 177, 211)',
              'surroundings': 'rgb(135, 197, 95)',
              'others / no fixation': 'rgb(203, 203, 203)'}

In [17]:
fig = go.Figure()
for target, target_df in df.groupby('target_maj'):
    fig.add_trace(go.Scattermapbox(
        lat=target_df.lat,
        lon=target_df.lon,
        text=target_df.target_maj,
        name=target,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=7, color=color_dict[target]),
    ))

fig.update_layout(
    autosize=True,
    showlegend=True,
    legend=dict(orientation='h'),
    hovermode='closest',
        mapbox=dict(
        accesstoken='YourMapBoxToken',
        center=dict(lat=52.2235, lon=6.8870),
        style='YourMapBoxStyle',
        zoom=17),
)
fig.show()

## Syncrhonize protocols and GPS

Using files:
- data/segment_protocol.tsv
- data/gps.csv

Syncrhonization logic: for each GPS point, associate with the protocol that is being verbalized at that moment

Result file: data/sync_protocol_gps.tsv

In [23]:
ta = pd.read_csv('data/segment_protocol.tsv', sep='\t')
gps = pd.read_csv('data/gps.csv')

# syncrhonize using your own timestamp marker
gps = gps.iloc[4:].reset_index()
# GPS[0] -> video start time : 119694
gps['timestamp'] = (gps['index'] - 4) # 0 second

# find a ta setence for each gps point
gps['content'] = ''
for i in ta.index:
    try: 
        gps_index_start = gps[gps['timestamp'].ge(ta.at[i, 'start_time'])].index[0]
        gps_index_end = gps[gps['timestamp'].le(ta.at[i, 'end_time'])].index[-1]
        for j in range (gps_index_start, gps_index_end+1):
            gps.at[j, 'content'] = ta.at[i, 'content']
    except:
        gps.at[j, 'content'] = ''

gps.to_csv('data/sync_protocol_gps.tsv', sep='\t', index=False)

## Plot fixation, protocol and GPS on a map
Using file:
- data/sync_protocol_gps.tsv
- data/sync_fixation_gps.tsv  

Protocols can be interactivly explored when moving the mouse onto the points

**You will need a mapbox token and style file for this**

In [25]:
et_gps = pd.read_csv('data/sync_fixation_gps.tsv', sep='\t')
ta_gps = pd.read_csv('data/sync_protocol_gps.tsv', sep='\t')
df = et_gps.merge(ta_gps, how='inner', 
                         left_on=['lon', 'lat'], right_on=['lon', 'lat'])
df.head()

,index_x,lat,lon,time_x,timestamp_x,start,end,new_target,target_count,target_maj,index_y,time_y,timestamp_y,content
0,4,52.223773,6.886489,2020-04-27T11:20:58Z,119694,119194,120194,surroundings,12,surroundings,4,2020-04-27T11:20:58Z,0,NaN
1,5,52.223773,6.886489,2020-04-27T11:20:59Z,120694,120194,121194,"['map_ar', 'surroundings']",2,map_ar,5,2020-04-27T11:20:59Z,1,NaN
2,6,52.223773,6.886489,2020-04-27T11:21:00Z,121694,121194,122194,['map_ar'],1,map_ar,6,2020-04-27T11:21:00Z,2,so I'm confused right now.
3,7,52.223773,6.886489,2020-04-27T11:21:01Z,122694,122194,123194,[],0,NaN,7,2020-04-27T11:21:01Z,3,What this means.
4,8,52.223773,6.886489,2020-04-27T11:21:02Z,123694,123194,124194,['map_ar'],1,map_ar,8,2020-04-27T11:21:02Z,4,What this means.


In [28]:
# plot using the same color dict
df.target_maj.replace(['others', 'No fixation'], 'others / no fixation', inplace=True)
fig = go.Figure()
for target, target_df in df.groupby('target_maj'):
    fig.add_trace(go.Scattermapbox(
        lat=target_df.lat,
        lon=target_df.lon,
        text=target_df.content,
        name=target,
        hoverinfo='text',
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=7, color=color_dict[target]),
    ))

fig.update_layout(
    autosize=True,
    showlegend=True,
    legend=dict(orientation='h'),
    hovermode='closest',
        mapbox=dict(
        accesstoken='Your MapBox Token',
        center=dict(lat=52.2235, lon=6.8870),
        style='Your Style',
        zoom=17),
)
fig.show()